# Few-shot JERX task offline inference

In [ ]:
#|default_exp jerx.offline.llm

In [ ]:
#|hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#|export
from pathlib import Path
import pandas as pd
from bellek.jerx.utils import parse_triplets
from bellek.logging import get_logger

log = get_logger(__name__)

In [ ]:
#|hide
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# |export

def parse_triplet_response(response: str, *args, **kwargs) -> list[tuple[str, str, str]]:
    triplets = parse_triplets(response.strip())
    return [(e1, rel, e2) if e1 != e2 else (e1, rel, e2 + "(obj)") for e1, rel, e2 in triplets]


def make_kg_triplet_extract_fn(inference_cache_filepath: Path):
    df = pd.read_json(inference_cache_filepath, lines=True, orient="records")
    df.set_index("text", inplace=True)

    def extract_kg_triplets(text: str) -> list[tuple[str, str, str]]:
        return parse_triplet_response(df.loc[text]['generation'])

    return extract_kg_triplets

In [ ]:
!pwd

/Users/bdsaglam/dev/repos/bellek/nbs


In [ ]:
df = pd.read_json('/Users/bdsaglam/dev/repos/bellek/data/generated/musique-kg-llm/train/jerx-inferences.jsonl', lines=True)
df.set_index("text", inplace=True)
df

,id,paragraph_idx,generation
text,,,
"# Midland, Texas\nMidland is a city in and the county seat of Midland County, Texas, United States, on the Southern Plains of the state's western area. A small portion of the city extends into Martin County.",2hop__128801_205185,1,"Midland, Texas | is part of | Midland County, ..."
"# KNFM\nKNFM (92.3 FM), branded as ""Lonestar 92"", is a Country music formatted radio station that serves the Midland–Odessa metropolitan area. The station broadcasts on FM frequency 92.3 and is under ownership of Townsquare Media.",2hop__128801_205185,11,KNFM | owner | Townsquare Media\nKNFM | broadc...
"# KQRX\nKQRX (95.1 FM, ""Rock 95.1"") is a commercial radio station located in Midland, Texas, broadcasting to the Midland-Odessa market. KQRX airs an active rock music format.",2hop__128806_205185,5,KQRX | broadcast area | Midland–Odessa\nKQRX |...
"# Atlantic City, New Jersey\nThere were 15,504 households, of which 27.3% had children under the age of 18 living with them, 25.9% were married couples living together, 22.2% had a female householder with no husband present, and 44.8% were non-families. 37.5% of all households were made up of individuals, and 14.3% had someone living alone who was 65 years of age or older. The average household size was 2.50 and the average family size was 3.34.",2hop__128895_11424,3,"Atlantic City, New Jersey | area of households..."
"# WPUR\nWPUR (107.3 FM) is a country music formatted radio station in Atlantic City, New Jersey. WPUR is more commonly known as ""Cat Country 107.3"". Its transmitter is located in Atlantic City, while studios are in Northfield, New Jersey.",2hop__128895_11424,12,"WPUR (107.3 FM) | location | Atlantic City, Ne..."
...,...,...,...
"# Uffizi\nThe building of Uffizi complex was begun by Giorgio Vasari in 1560 for Cosimo I de' Medici so as to accommodate the offices of the Florentine magistrates, hence the name , ""offices"". The construction was later continued by Alfonso Parigi and Bernardo Buontalenti; it was completed in 1581. The top floor was made into a gallery for the family and their guests and included their collection of Roman sculptures.",2hop__651488_94210,12,Uffizi | architect | Alfonso Parigi\nUffizi | ...
"# Pietà (Perugino)\nPietà is a painting by the Italian Renaissance artist Pietro Perugino, executed around 1483-1493, and housed in the Uffizi Gallery, Florence.",2hop__651488_94210,18,Pietà (Perugino) | creator | Pietro Perugino\n...
"# Jim Norton (defensive lineman)\nJames Alfred Norton (born November 18, 1942 in Wilmington, North Carolina) is a former American football defensive lineman in the National Football League for the San Francisco 49ers, Atlanta Falcons, Philadelphia Eagles, Washington Redskins, and New York Giants. He played college football at the University of Washington and was drafted in the third round of the 1965 NFL Draft.",2hop__525596_543261,18,James Alfred Norton (American football) | coll...


In [ ]:
func = make_kg_triplet_extract_fn('/Users/bdsaglam/dev/repos/bellek/data/generated/musique-kg-llm/train/jerx-inferences.jsonl')
func(df.index[0])

[('Midland, Texas', 'is part of', 'Midland County, Texas'),
 ('Midland, Texas', 'is part of', 'Texas'),
 ('Midland, Texas', 'is part of', 'Martin County, Texas'),
 ('Texas', 'has to its west', 'Southern Plains'),
 ('Texas', 'country', 'United States')]

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()